In [1]:
# generic tools
import numpy as np
import cv2

# tools from sklearn
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# tools from tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import cifar10

# matplotlib
import matplotlib.pyplot as plt

2023-03-17 14:21:53.471585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train_grey = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_train])
X_test_grey = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_test])


X_train_scaled = (X_train_grey)/255.0
X_test_scaled = (X_test_grey)/255.0

nsamples, nx, ny = X_train_scaled.shape
X_train_dataset = X_train_scaled.reshape((nsamples,nx*ny))

nsamples, nx, ny = X_test_scaled.shape
X_test_dataset = X_test_scaled.reshape((nsamples,nx*ny))

In [13]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [25]:
# define architecture 784x256x128x10
model = Sequential()#sequential betyder feedforward model
model.add(Dense(256, 
                input_shape=(1024,), 
                activation="relu"))#her adder man et layer, der er fully connected, 256 er nodes, 784 er input
model.add(Dense(128, 
                activation="relu"))# endnu et layer, modellen finder selv input her
model.add(Dense(10, 
                activation="softmax"))

In [26]:
# train model using SGD
sgd = SGD(0.01)
model.compile(loss="categorical_crossentropy", 
              optimizer=sgd, 
              metrics=["accuracy"])

In [27]:
history = model.fit(X_train_dataset, y_train, 
                    epochs=10, 
                    batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 7s 4ms/step - loss: 2.0483 - accuracy: 0.2593
Epoch 2/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.9199 - accuracy: 0.3200
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.8479 - accuracy: 0.3467
Epoch 4/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.7980 - accuracy: 0.3656
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.7592 - accuracy: 0.3811
Epoch 6/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.7260 - accuracy: 0.3915
Epoch 7/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6970 - accuracy: 0.4033
Epoch 8/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6721 - accuracy: 0.4122
Epoch 9/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6470 - accuracy: 0.4224
Epoch 10/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.625

In [28]:
# evaluate network
print("[INFO] evaluating network...")
predictions = model.predict(X_test_dataset, batch_size=32)

[INFO] evaluating network...
313/313 [==============================] - 1s 2ms/step


In [29]:
print(classification_report(y_test.argmax(axis=1), # 
                            predictions.argmax(axis=1), 
                            target_names=[str(x) for x in lb.classes_]))

              precision    recall  f1-score   support

           0       0.49      0.36      0.41      1000
           1       0.35      0.62      0.45      1000
           2       0.35      0.28      0.31      1000
           3       0.31      0.20      0.24      1000
           4       0.40      0.24      0.30      1000
           5       0.41      0.32      0.36      1000
           6       0.41      0.34      0.37      1000
           7       0.47      0.40      0.43      1000
           8       0.49      0.52      0.50      1000
           9       0.33      0.64      0.43      1000

    accuracy                           0.39     10000
   macro avg       0.40      0.39      0.38     10000
weighted avg       0.40      0.39      0.38     10000

